In [ ]:
import collections
import os
import json
from difflib import SequenceMatcher as SM
from itertools import combinations
from datetime import date
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time

In [ ]:
working_forder = '/Users/luisa/Desktop/data_grant/' # The data_grant folder

In [ ]:
pubs_when_grant_primary = collections.defaultdict(dict)
pubs_when_grant_secondary_tracker = collections.defaultdict(dict)
pubs_when_grant_secondary = collections.defaultdict(dict)

fname_grant_jsons = os.listdir("/Users/luisa/Desktop/data_grant")
for fname_json in fname_grant_jsons:
    with open("/Users/luisa/Desktop/data_grant/" + fname_json, "r") as fp:
        dic_pergrant = json.load(fp)
        pubs = dic_pergrant.keys()
        grant_name = fname_json[:-5]
        pubs_when_grant_primary[grant_name] = collections.defaultdict(dict)
        pubs_when_grant_secondary[grant_name] = collections.defaultdict(dict)
        for pub in pubs:
            authors = "; ".join(dic_pergrant[pub]["authors"])
            title = dic_pergrant[pub]["title"]
            link = dic_pergrant[pub]["link"]
            date = dic_pergrant[pub]["date"]
            other_ids = ";".join(dic_pergrant[pub]["other_ids"])
            primary_awards = ";".join(dic_pergrant[pub]["primary_awards"]) if len(dic_pergrant[pub]["primary_awards"]) > 1 else " -- "
            secondary_awards = ";".join(dic_pergrant[pub]["secondary_awards"])
            contact_4DN_author = dic_pergrant[pub]["4DN_member_author"]
            added_date = dic_pergrant[pub]["added"]

            pubs_when_grant_primary[grant_name][pub]['title'] = title
            pubs_when_grant_primary[grant_name][pub]['authors'] = authors
            pubs_when_grant_primary[grant_name][pub]['date'] = added_date
            pubs_when_grant_primary[grant_name][pub]['link'] = link
            pubs_when_grant_primary[grant_name][pub]['other_ids'] = other_ids
            pubs_when_grant_primary[grant_name][pub]['primary_awards'] = primary_awards
            pubs_when_grant_primary[grant_name][pub]['secondary_awards'] = secondary_awards
            pubs_when_grant_primary[grant_name][pub]['date_added'] = added_date
            pubs_when_grant_primary[grant_name][pub]['contact_4DN_author'] = contact_4DN_author
            
#     # Keeps track of the grants of the publications secondary awards
    
    for grant in pubs_when_grant_primary:
        for pub in pubs_when_grant_primary[grant]:
            if pubs_when_grant_primary[grant][pub]['secondary_awards'] != '':
                if ';' not in pubs_when_grant_primary[grant][pub]['secondary_awards']:
                    sec_award = pubs_when_grant_primary[grant][pub]['secondary_awards']
                    pubs_when_grant_secondary[sec_award][pub] = pubs_when_grant_primary[grant][pub]
                else:
                    sec_award_list = pubs_when_grant_primary[grant][pub]['secondary_awards'].split(';')
                    for sec_award in sec_award_list:
                        pubs_when_grant_secondary[sec_award][pub] = pubs_when_grant_primary[grant][pub]

In [ ]:
# Accesses to the google sheets
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)
book_id = '1PqqMQn7R0qbYm0bUOf3LwCtPJryIbo7YoRf2_PZxOQY' # This is the test spreadsheet
test_spreadsheet = gc.open_by_key(book_id)

In [ ]:
worksheet = test_spreadsheet.worksheet('Publication List')

In [ ]:
worksheet_values = worksheet.get_all_values()

In [ ]:
import pandas as pd
df = pd.DataFrame(worksheet_values[1:], columns=worksheet_values[0])

In [ ]:
from collections import OrderedDict
google_sheet = df.to_dict(orient='records', into=OrderedDict)

In [ ]:
x = 0
for item in google_sheet:
    print(item)
    x = x + 1
    if x == 1:
        break

In [ ]:
dekker_grant = pubs_when_grant_primary["U54DK107980"] # Test a specific grant

In [ ]:
print(len(dekker_grant))

In [ ]:
x = 0
for item in dekker_grant.values():
    print(item)
    x = x + 1
    if x == 1:
        break

In [ ]:
# Comparing with existing google sheets

In [ ]:
new_sheet = []
for a_row in dekker_grant.values():
    #check excel
    existing_item = [i for i in google_sheet if a_row['title'] == i['Title']]
    if existing_item:
        new_sheet.append(existing_item)
    else:
        print('No item')

In [ ]:
for i in google_sheet:
    for key, val in i.items():
        #print(key, val)
        break
    

In [ ]:
for i in new_sheet:
    print(i)
    break

In [ ]:
# Adding secondary awards below

In [ ]:
pubs_when_grant_secondary['EB021236'] 

In [ ]:
new_worksheet = test_spreadsheet.add_worksheet('Test Results', 100, 15)

In [ ]:
gs_write = []
row = 1
for r,line in enumerate(new_sheet):
    print(line)
    row = r + 1
    
    if row == 1:
        for c, key in enumerate(line[0]):
            print()
            print(key)
            
            col = c + 1
            gs_write.append(gspread.models.Cell(row, col, key))
    row = r + 2
    for c, key in enumerate(line[0]):
        col = c + 1
        gs_write.append(gspread.models.Cell(row, col, line[0][key]))
    new_worksheet.update_cells(gs_write)

In [ ]:
for line in new_sheet:
    print(line)
    for item in line:
        print(item.keys())
        break
    break

In [ ]:
# Adding all the data to the spreadsheet at once
gs_write = [] # A list of cell objects to write to the worksheet
# The data that is going to be written in the worksheet. In this example, each key is a column.

#Writting the data to the list
row = 1
for r,line in enumerate(user_list):
    row = r + 1
    
    # write columns
    if row == 1:
        for c, key in enumerate(line[0]):
            col = c + 1
            gs_write.append(gspread.models.Cell(row, col, key)) 
    row = r + 2
    # write values
    for c, key in enumerate(line):
        col = c + 1
        gs_write.append(gspread.models.Cell(row, col, line[key])) 
    
# #Write the cells to the worksheet
#worksheet.update_cells(gs_write)

In [ ]:
print()